In [ ]:
import json
import re
import boto3
from math import ceil

# Configuración AWS (asegúrate de tener credenciales configuradas en tu máquina)
polly = boto3.client("polly", region_name="us-east-1")

# Archivo JSON de entrada
JSON_FILE = "../summaries-local/2025-08-15/iglesia.json"

# Carpeta donde guardar los MP3
OUTPUT_FOLDER = "audio/"

# Leer JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)


def split_text(text, max_length=3000):
    """
    Divide el texto en fragmentos que no superen el límite de longitud.
    """
    # Calcula el número de fragmentos necesarios
    num_chunks = ceil(len(text) / max_length)
    return [text[i * max_length : (i + 1) * max_length] for i in range(num_chunks)]


def synthesize_speech(text):
    """
    Sintetiza el texto en fragmentos que cumplen con los límites de longitud.
    """
    chunks = split_text(text)
    audio_streams = []

    for chunk in chunks:
        try:
            response = polly.synthesize_speech(
                Text=chunk,
                LanguageCode="es-ES",
                OutputFormat="mp3",
                VoiceId="Raul",  # voz española femenina, puedes cambiar a "Conchita"
                Engine="long-form",  # mejor calidad
            )
            audio_streams.append(response["AudioStream"].read())
        except polly.exceptions.TextLengthExceededException as e:
            print(f"Error: {e}")
            break

    # Combina los fragmentos de audio en un solo flujo
    audio_data = b"".join(audio_streams)
    return audio_data

for idx in range(len(data["titulo"])):
    titulo = data["titulo"][str(idx)]
    texto = data["texto"][str(idx)]
    filename = data["filename"][str(idx)] + ".mp3"

    # 1) Cortar desde "queridos..."
    match = re.search(r"(queridos.*)", texto, flags=re.IGNORECASE | re.DOTALL)
    if match:
        texto = match.group(1)

    # 2) Eliminar números y paréntesis
    texto = re.sub(r"\([^)]*\)", "", texto)  # elimina contenido entre paréntesis
    texto = re.sub(r"\d+", "", texto)        # elimina dígitos restantes
    texto = texto.strip()
    # 3) Eliminar espacios dobles y limpiar
    texto = re.sub(r"\s+", " ", texto).strip()
    texto = re.split(r"Saludos", texto, flags=re.IGNORECASE)[0]
    texto = re.split(r"Después del Ángelus", texto, flags=re.IGNORECASE)[0]
    texto = re.sub(r"_", "", texto).strip()

    # 4) Generar audio con AWS Polly
    audio_data = synthesize_speech(texto)

    # Guardar el archivo MP3
    with open(OUTPUT_FOLDER + filename, "wb") as f_audio:
        f_audio.write(audio_data)

    print(f"✅ Audio generado: {filename}")


✅ Audio generado: 2025-08-13_audiencia_audiencia_general_del_13_de_agosto_de_2025_ciclo_de_catequesis_jubileo_2025_jesucristo_nuestra_esper.mp3
✅ Audio generado: 2025-08-10_ángelus_angelus_10_de_agosto_de_2025.mp3


In [ ]:
import json
import re
import os
import azure.cognitiveservices.speech as speechsdk

# --- Configuración de Azure ---
# Es recomendable usar variables de entorno para no exponer las claves en el código.
# CLAVE_AZURE = os.environ.get("AZURE_SPEECH_KEY")
# REGION_AZURE = os.environ.get("AZURE_SPEECH_REGION")
CLAVE_AZURE = ""
REGION_AZURE = "eastus" # ej: "westeurope"

# --- Rutas de archivos (sin cambios) ---
JSON_FILE = "../summaries-local/2025-08-15/iglesia.json"
OUTPUT_FOLDER = "audio/"

# --- Lectura del JSON (sin cambios) ---
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

def synthesize_speech_azure(text, output_filename):
    """
    Sintetiza el texto usando Azure y lo guarda directamente en un archivo.
    Azure maneja textos largos de forma nativa, no es necesario dividirlos.
    """
    # 1. Configurar la conexión con la clave y la región.
    speech_config = speechsdk.SpeechConfig(subscription=CLAVE_AZURE, region=REGION_AZURE)
    
    # 2. Configurar la salida de audio directamente al archivo deseado.
    audio_config = speechsdk.audio.AudioOutputConfig(filename=output_filename)

    # 3. Elegir la voz. 'es-ES-TeoNeural' es una voz masculina en español de alta calidad.
    speech_config.speech_synthesis_voice_name = 'es-ES-TeoNeural'

    # 4. Crear el sintetizador.
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    # 5. Realizar la síntesis. El SDK gestiona el texto largo por ti.
    result = speech_synthesizer.speak_text_async(text).get()

    # 6. Comprobar el resultado.
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print(f"✅ Audio generado: {os.path.basename(output_filename)}")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print(f"❌ Síntesis de voz cancelada: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"Detalles del error: {cancellation_details.error_details}")

# --- Bucle principal de procesamiento (lógica de limpieza sin cambios) ---
for idx in [0]: #range(len(data["titulo"]))
    titulo = data["titulo"][str(idx)]
    texto = data["texto"][str(idx)]
    filename_mp3 = data["filename"][str(idx)] + ".mp3"
    output_path = os.path.join(OUTPUT_FOLDER, filename_mp3)

    # 1) Cortar desde "queridos..." (sin cambios)
    match = re.search(r"(queridos.*)", texto, flags=re.IGNORECASE | re.DOTALL)
    if match:
        texto = match.group(1)

    # 2) Eliminar números y paréntesis (sin cambios)
    texto = re.sub(r"\([^)]*\)", "", texto)
    texto = re.sub(r"\d+", "", texto)
    texto = texto.strip()
    
    # 3) Limpiar espacios y cortar texto no deseado (sin cambios)
    texto = re.sub(r"\s+", " ", texto).strip()
    texto = re.split(r"Saludos", texto, flags=re.IGNORECASE)[0]
    texto = re.split(r"Después del Ángelus", texto, flags=re.IGNORECASE)[0]
    texto = re.sub(r"_", "", texto).strip()

    # 4) Generar audio con la nueva función de Azure
    # La función ahora se encarga de todo, incluida la escritura del archivo.
    synthesize_speech_azure(texto, output_path)

✅ Audio generado: 2025-08-13_audiencia_audiencia_general_del_13_de_agosto_de_2025_ciclo_de_catequesis_jubileo_2025_jesucristo_nuestra_esper.mp3


In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk

# --- Configuración de Azure ---
CLAVE_AZURE = ""
REGION_AZURE = "eastus" # ej: "westeurope"

def synthesize_speech_azure(texto, output_filename, voice_name="es-ES-ElviraNeural", speed_percentage=0):
    """
    Sintetiza texto usando SSML para controlar la voz y la velocidad.

    Args:
        texto (str): El texto a convertir en voz.
        output_filename (str): La ruta donde se guardará el archivo MP3.
        voice_name (str): El nombre corto de la voz de Azure a usar (ej: 'es-ES-TeoNeural').
        speed_percentage (int): El ajuste de velocidad. 0 es normal, -20 es 20% más lento, 50 es 50% más rápido.
    """
    speech_config = speechsdk.SpeechConfig(subscription=CLAVE_AZURE, region=REGION_AZURE)
    audio_config = speechsdk.audio.AudioOutputConfig(filename=output_filename)
    
    # NO establecemos la voz aquí, lo haremos a través de SSML
    # speech_config.speech_synthesis_voice_name = voice_name
    
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    # Construir el string SSML
    ssml_string = f"""
        <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='es-ES'>
            <voice name='{voice_name}'>
                <prosody rate='{speed_percentage}%'>
                    {texto}
                </prosody>
            </voice>
        </speak>
    """
    
    # Usar 'speak_ssml_async' en lugar de 'speak_text_async'
    result = speech_synthesizer.speak_ssml_async(ssml_string).get()

    # Comprobación de errores (sin cambios)
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print(f"✅ Audio '{os.path.basename(output_filename)}' generado con la voz '{voice_name}' y velocidad '{speed_percentage}%'.")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print(f"❌ Síntesis de voz cancelada: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"Detalles del error: {cancellation_details.error_details}")

# --- EJEMPLOS DE USO ---

texto_ejemplo = "Puedes probar diferentes voces y velocidades para encontrar la combinación perfecta para tu contenido."

# Ejemplo 1: Voz masculina a velocidad normal
synthesize_speech_azure(
    texto_ejemplo,
    "salida_teo_normal.mp3",
    voice_name="es-ES-TeoNeural",
    speed_percentage=-10
)



✅ Audio 'salida_teo_normal.mp3' generado con la voz 'es-ES-TeoNeural' y velocidad '-10%'.


In [7]:
import azure.cognitiveservices.speech as speechsdk

# --- Configuración de Azure ---
CLAVE_AZURE = "TU_CLAVE_DE_AZURE"
REGION_AZURE = "TU_REGION_DE_AZURE"

def listar_voces_espanol():
    """Se conecta a Azure y lista todas las voces disponibles para 'es-ES'."""
    speech_config = speechsdk.SpeechConfig(subscription=CLAVE_AZURE, region=REGION_AZURE)
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)
    
    print("Obteniendo lista de voces para español (es-ES)...")
    result = speech_synthesizer.get_voices_async(locale="es-ES").get()
    
    print("--- Voces Disponibles en Español ---")
    for voice in result.voices:
        # Imprime el nombre corto (ej: 'es-ES-ElviraNeural'), género y nombre local.
        print(f"- Nombre: {voice.short_name}, Género: {voice.gender.name}, Local: {voice.local_name}")
    print("---------------------------------")

# Ejecuta la función para ver la lista
listar_voces_espanol()

Obteniendo lista de voces para español (es-ES)...
--- Voces Disponibles en Español ---
---------------------------------
